In [322]:
from sklearn import datasets    # Importing Datasets module from sklearn for Iris Dataset

import pandas as pd    # Pandas library to handle iris dataset as Dataframe

import numpy as np    # Numpy library to handle arrays and use numpy methods

from sklearn.tree import export_graphviz   # export tree to graph

import pydotplus    # pyplotplus library to export graph as pdf

In [323]:
iris = datasets.load_iris()    # load_iris method to get iris dataset from sklearn

iris_df = pd.DataFrame(iris.data,columns=['SL','SW','PL','PW'])    # converting iris data into Dataframe and adding headers

iris_df['target']=iris.target    # combining target values of iris dataset with Dataframe

In [324]:
from sklearn import model_selection
# Seperating training and testing data for later checking accuracy of decision tree
X_train,X_test,Y_train,Y_test = model_selection.train_test_split(iris.data,iris.target,random_state = 42)

# combining target values of iris with features data for training data
training_data = np.concatenate([X_train,Y_train.reshape(-1,1)],axis=1) 

# combining target values of iris with features data for testing data
testing_data = np.concatenate([X_test,Y_test.reshape(-1,1)],axis=1)

In [325]:
target_names = iris.target_names   # setting target_names variable as target_names from iris dataset

feature_names = list(iris.feature_names)    # setting feature_names variable as feature_names from iris dataset

In [326]:
iris_df.describe()    # Describing iris dataset to check if null values are present and get info about dataset

,SL,SW,PL,PW,target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [327]:
class Question:
    """A Question is used to partition a dataset.

    This class just records a 'column number' (e.g., 0 for SL or sepal length in cm) and a
    'column value' (e.g., 7.9). The 'match' method is used to compare
    the feature value in an example to the feature value stored in the
    question.
    """
    def __init__(self,column,value):
        self.column = column
        self.value = value
    
    def match(self, example):
        # Compare the feature value in an example to the
        # feature value in this question.
        # we are checking if value in example is less than or equal to value set in question
        val = example[self.column]
        return val <= self.value
    
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        return "Is %s <= %s?"% (feature_names[int(self.column)], self.value)

In [328]:
def partition(rows,question):
    """Partitions a dataset.

    For each row in the dataset, check if it matches the question. If
    so, add it to 'true rows', otherwise, add it to 'false rows'.
    """
    true_rows,false_rows = [],[]
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows,false_rows

In [329]:
def entropy(target):
    """Calculate the Entropy for a list of rows.

    Since target contains all columns. We need to retrive last column.
    Using np.unique function we can get unique values in our target_col and count of those unique values
    Then we can apply entropy formula for each unique result to get total entropy.
    """
    target_col = np.array(target)[:,-1]
    elements, counts = np.unique(target_col ,return_counts=True)
    entropy= 0
    for i in range(len(elements)):
        entropy+= (-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts))
    return entropy

In [330]:
def info_gain(left,right,parent):
    """Information Gain.

    Difference between Entropy of parent node and Average entropy of child nodes.
    """
    nl = len(left)
    nr = len(right)
    np = len(parent)
    avgEntropy = (nl/np)*entropy(left) + (nr/np)*entropy(right)
    return entropy(parent) - avgEntropy

In [331]:
def find_best_split(rows):
    """Find the best question to ask by iterating over every feature / value
    and calculating the information gain."""
    
    best_gain = 0    # keep track of the best information gain
    
    best_question = None    # keep train of the feature / value that produced it
    
    n_features = len(rows[0])-1    # number of columns
    
    for col in range(n_features):   # for each feature
        
        values = set([row[col] for row in rows])  # unique values in the column
        
        for val in values:  # for each value
            
            question = Question(col,val)
            
            # try splitting the dataset
            true_rows, false_rows = partition(rows,question)

            # Skip this split if it doesn't divide the
            # dataset.
            if(len(true_rows) == 0 or len(false_rows) ==0 ):
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, rows)

            # We can use > instead of >= to compare best_gain and gain.
            if(gain >= best_gain):
                best_gain, best_question = gain, question
        
    return best_gain, best_question

In [332]:
def predictions(rows):
    np_rows = np.array(rows)
    elements, counts = np.unique(np_rows[:,-1] ,return_counts=True)
    predictions={}
    for (e,c) in zip(elements,counts):
        predictions[int(e)]=c
    return predictions

In [333]:
class Leaf:
    """A Leaf node classifies data.

    This holds a dictionary of class (e.g., "Setosa") -> number of times
    it appears in the rows from the training data that reach this leaf.
    """
    def __init__(self,rows):
        self.predictions = predictions(rows)
        self.rows = rows
            

In [334]:
class Decision_Node:
    """A Decision Node asks a question.

    This holds a reference to the question, and to the two child nodes.
    """
    def __init__(self,question,true_branch,false_branch,rows,gain):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch
        self.rows = rows
        self.gain = gain
        
    def export(self,file_name):
        dot_data = export_graphviz(self, out_file=None)
        graph = pydotplus.graph_from_dot_data(dot_data)
        graph.write_pdf(file_name+".pdf")

In [335]:
def build_tree(rows):
    """Builds the tree.

    This uses recursion to get to Leaf Nodes where split is not possible and our gain turns to 0.
    Once we get to Leaf Node for both true and false outputs we convert into branches
    Then add it to Decision_Node Object to build a decision tree.
    """
    
    # Try partitioing the dataset on each of the unique attribute,
    # calculate the information gain,
    # and return the question that produces the highest gain.
    gain, question = find_best_split(rows)
    
    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf
    if gain == 0:
        return Leaf(rows)
    
    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)
    
    # Recursively build the true branch.
    true_branch = build_tree(true_rows)
    
    # Recursively build the false branch.
    false_branch = build_tree(false_rows)
    
    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question,true_branch,false_branch,rows,gain)

In [336]:
def print_tree(node,feature_names,target_names, level=0):
    """Tree printing function"""
    
    print("\nLevel",level)
    pred = predictions(node.rows)
    for key in pred:
        print( "Count of",target_names[key], "=", pred[key])
    print("Current Entropy  is =",entropy(node.rows))
    
    # Base case: we've reached a leaf
    if( isinstance(node,Leaf) ):
        print("Reached leaf Node")
        return
    
    col = int(node.question.column)
    print("Splitting on feature",feature_names[col],"with gain ratio",node.gain)

    # Call this function recursively on the true branch
    print_tree(node.true_branch,feature_names,target_names,level+1)
    
    # Call this function recursively on the false branch
    print_tree(node.false_branch,feature_names,target_names,level+1)

In [337]:
OR_data = [[1,1,1],
           [0,1,1],
           [1,0,1],
           [0,0,0]]
OR_feature_names = ['X1','X2']
OR_target_names = [0,1]

In [338]:
OR_tree = build_tree(OR_data)

In [339]:
print_tree(OR_tree,OR_feature_names,OR_target_names)


Level 0
Count of 0 = 1
Count of 1 = 3
Current Entropy  is = 0.8112781244591328
Splitting on feature X2 with gain ratio 0.31127812445913283

Level 1
Count of 0 = 1
Count of 1 = 1
Current Entropy  is = 1.0
Splitting on feature X1 with gain ratio 1.0

Level 2
Count of 0 = 1
Current Entropy  is = 0.0
Reached leaf Node

Level 2
Count of 1 = 1
Current Entropy  is = 0.0
Reached leaf Node

Level 1
Count of 1 = 2
Current Entropy  is = 0.0
Reached leaf Node


In [340]:
def export_graph(obj,file_name,feature_names):
        dot_data = export_graphviz(obj, out_file=None,feature_names=feature_names)
        graph = pydotplus.graph_from_dot_data(dot_data)
        graph.write_pdf(file_name+".pdf")
# export_graph(OR_tree,"OR_Tree")

In [341]:
#  Builiding tree of training data of iris dataset.
# my_tree = build_tree(training_data) 

# To use whole iris dataset to build tree pass `iris_df.values` instead of `training_data` OR vice-versa
iris_tree = build_tree(iris_df.values) 

target_names = iris.target_names   # setting target_names variable as target_names from iris dataset

feature_names = list(iris.feature_names)    # setting feature_names variable as feature_names from iris dataset

In [342]:
print_tree(iris_tree,feature_names,target_names)


Level 0
Count of setosa = 50
Count of versicolor = 50
Count of virginica = 50
Current Entropy  is = 1.584962500721156
Splitting on feature petal width (cm) with gain ratio 0.9182958340544894

Level 1
Count of setosa = 50
Current Entropy  is = 0.0
Reached leaf Node

Level 1
Count of versicolor = 50
Count of virginica = 50
Current Entropy  is = 1.0
Splitting on feature petal width (cm) with gain ratio 0.6901603707546748

Level 2
Count of versicolor = 49
Count of virginica = 5
Current Entropy  is = 0.44506485705083865
Splitting on feature petal length (cm) with gain ratio 0.21317043093799645

Level 3
Count of versicolor = 47
Count of virginica = 1
Current Entropy  is = 0.1460942501201363
Splitting on feature petal width (cm) with gain ratio 0.1460942501201363

Level 4
Count of versicolor = 47
Current Entropy  is = 0.0
Reached leaf Node

Level 4
Count of virginica = 1
Current Entropy  is = 0.0
Reached leaf Node

Level 3
Count of versicolor = 2
Count of virginica = 4
Current Entropy  is = 

In [343]:
from sklearn.tree import DecisionTreeClassifier

In [344]:
OR_clf = DecisionTreeClassifier(random_state=0,criterion='entropy')
train = np.array(OR_data)
OR_clf.fit(train[:,:-1],train[:,-1])

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [345]:
export_graph(clf,"OR_tree",OR_feature_names)

In [346]:
iris_clf = DecisionTreeClassifier(random_state=0,criterion='entropy')
iris_clf.fit(iris.data,iris.target)

DecisionTreeClassifier(criterion='entropy', random_state=0)

In [347]:
export_graph(iris_clf,"iris_tree",feature_names)